In [15]:
import os
import pathlib
import json
import osc_ingest_trino as osc
import trino
import sqlalchemy
from sqlalchemy.engine import create_engine
import pandas as pd
from pyarrow import fs, csv, parquet


def _do_sql(sql, engine, verbose=False):
    if type(sql) != sqlalchemy.sql.elements.TextClause:
      #  sql = sqlalchemy.sql.text(str(sql))
         sql = sql
    if verbose:
        print(sql)
    qres = engine.execute(sql)
    res = None
    if qres.returns_rows:
        res = qres.fetchall()
        if verbose:
            print(res)
    return res

MINIO_ACCESS_KEY = os.getenv('MINIO_ACCESS_KEY')
MINIO_SECRET_KEY = os.getenv('MINIO_SECRET_KEY')
s3_bucket = 'pcaf'
ingest_catalog = 'hive'
ingest_schema = 'pcaf'
sqlstring = f"trino://admin:@trino:8080/{ingest_catalog}"
engine = create_engine(sqlstring, connect_args={"http_scheme": "http"},)
engine.connect()

minio = fs.S3FileSystem( endpoint_override= 'minio:9000',
                     access_key=MINIO_ACCESS_KEY,
                     secret_key=MINIO_SECRET_KEY,
                     scheme='http')



In [16]:
sql=f"""
show tables from hive.pcaf
"""

_do_sql(sql, engine)

[('annexi',),
 ('non_annexi',),
 ('oecd',),
 ('pcaf_primap',),
 ('pcaf_primap_staging',),
 ('pcaf_unfccc_annexi_staging',),
 ('pcaf_unfccc_nonannexi_staging',),
 ('pcaf_unfccc_sovereign_emissions',),
 ('pcaf_unfccc_with_lulucf',),
 ('pcaf_unfccc_without_lulucf',),
 ('pcaf_wdi',),
 ('pcaf_wdi_staging',),
 ('primap',),
 ('worldbank',)]

In [17]:
sql=f"""
select * from {ingest_catalog}.{ingest_schema}.pcaf_unfccc_sovereign_emissions e
"""

#_do_sql(sql, engine)
pd.read_sql(sql, engine)

,country_iso_code,validity_date,ghg_total_with_lulucf,ghg_total_with_lulucf_units,ghg_total_without_lulucf,ghg_total_without_lulucf_units,scope1_excl_source,gdp,gdp_units,gdp_ppp,gdp_ppp_units,ghg_intensity_with_lulucf_per_gdp,ghg_intensity_with_lulucf_per_gdp_units,ghg_intensity_without_lulucf_per_gdp,ghg_intensity_without_lulucf_per_gdp_units
0,RUS,1990,3.089163e+06,kt CO2e,3.166579e+06,kt CO2e,UNFCCC,5.168143e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
1,ROU,1990,2.285332e+05,kt CO2e,2.571372e+05,kt CO2e,UNFCCC,3.824433e+10,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
2,NOR,1991,3.672341e+04,kt CO2e,4.833904e+04,kt CO2e,UNFCCC,1.218717e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
3,NOR,2011,2.882927e+04,kt CO2e,5.399915e+04,kt CO2e,UNFCCC,5.013605e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
4,NOR,2020,3.004196e+04,kt CO2e,4.921464e+04,kt CO2e,UNFCCC,3.676334e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1402,IRL,1997,6.950554e+04,kt CO2e,6.368362e+04,kt CO2e,UNFCCC,8.285665e+10,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
1403,NLD,1993,2.367841e+05,kt CO2e,2.304643e+05,kt CO2e,UNFCCC,3.535502e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
1404,AUT,2011,6.714581e+04,kt CO2e,8.250576e+04,kt CO2e,UNFCCC,4.316852e+11,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
1405,FRA,2015,4.235809e+05,kt CO2e,4.602837e+05,kt CO2e,UNFCCC,2.439189e+12,USD,None,None,None,CO2eq * kt / USD,None,CO2eq * kt / USD
